In [13]:
#### Constants in the scheme
C = var("C", latex_name="\\mathcal{C}") ## Courant number
s1 = var("s1", latex_name="s_1")        ## First relaxation parameter (useless)
s2 = var("s2", latex_name="s_2")        ## Second relaxation parameter

assume(C,'real')
assume(s2,'real')

#### We make natural assumptions
assume(s2>0)
assume(s2<=2)

#### Pieces involved in the scheme
q = 2 ## Number of discrete velocities
c1 = 1  ## First discrete velocity
c2 = -1 ## Second discrete velocity
r = 1 ## Stencil to the left
p = 1 ## Stencil to the right
M = matrix([[1,  1], \
            [1, -1]]) ## Moment matrix
eps1 = 1 ## First equilibrium coefficient (useless)
eps2 = C ## Second equilibrium coefficient

#### Construction of the objects involved in the analysis
k = var("kappa", latex_name="\\kappa") 

Minv = M.inverse()
K = identity_matrix(q)+diagonal_matrix([s1, s2])*(matrix([[eps1], [eps2]])*matrix([[1, 0]])-identity_matrix(q))
Ehat = M*diagonal_matrix([k**(-c1), k**(-c2)])*Minv*K #### Bulk matrix scheme "Fourier"
for i in range(q):
    for j in range(q):
        Ehat[i, j] = Ehat[i, j].full_simplify().collect(k)
    
z = var("z", latex_name="z") 

charEq = (z*identity_matrix(q)-Ehat).determinant().full_simplify().collect(k) #### Characteristic equation
dm1 = charEq.coefficient(k, -1).collect(z) ### Coefficient d_{-1} of the characteristic equation
d0  = charEq.coefficient(k,  0).collect(z) ### Coefficient d_{0} of the characteristic equation
dp1 = charEq.coefficient(k,  1).collect(z) ### Coefficient d_{1} of the characteristic equation

Em1 = matrix(SR, q, q) ### Matrix for the point -1
E0  = matrix(SR, q, q) ### Matrix for the point 0
Ep1 = matrix(SR, q, q) ### Matrix for the point 1
for i in range(q):
    for j in range(q):
        Em1[i, j] = Ehat[i, j].coefficient(k, -1)
        E0[i, j]  = Ehat[i, j].coefficient(k,  0)
        Ep1[i, j] = Ehat[i, j].coefficient(k,  1)
Lz = - Em1 + (z*identity_matrix(q)-E0)*k - Ep1*k**2 ### Matrix polynomial in k
print('!! Sanity check: the result must be zero.')
pretty_print((Lz.determinant()/k**(r*q)-charEq).full_simplify()) ### Sanity check

kplus  = ((-d0+sqrt(d0**2-4*dp1*dm1))/2/dp1).full_simplify() ### Root of the char eq with the plus
kminus = ((-d0-sqrt(d0**2-4*dp1*dm1))/2/dp1).full_simplify() ### Root of the char eq with the minus


Pi = dm1/dp1 ### This is the product of the roots in k 

!! Sanity check: the result must be zero.


0

In [14]:
##### Boundary conditions that we analyze for any value of C and s_2
bdMat_BB = z*identity_matrix(q) - M*matrix([[0, 1], [0, k]])*Minv*K ## Bounce back
bdMat_ABB = z*identity_matrix(q) - M*matrix([[0, -1], [0, k]])*Minv*K ## Anti Bounce back
bdMat_TwoABB = z*identity_matrix(q) - M*matrix([[0, -k], [0, k]])*Minv*K ## Two steps Anti Bounce back
bdMat_sigma1 = z*identity_matrix(q) - M*matrix([[1, 0], [0, k]])*Minv*K ## Extrapolation sigma = 1
bdMat_sigma2 = z*identity_matrix(q) - M*matrix([[2-k, 0], [0, k]])*Minv*K ## Extrapolation sigma = 2
bdMat_sigma3 = z*identity_matrix(q) - M*matrix([[3-3*k+k**2, 0], [0, k]])*Minv*K ## Extrapolation sigma = 3
bdMat_sigma4 = z*identity_matrix(q) - M*matrix([[4-6*k+4*k**2-k**3, 0], [0, k]])*Minv*K ## Extrapolation sigma = 4
bdMat_kinDir = z*identity_matrix(q) - M*matrix([[0, 0], [0, k]])*Minv*K ## Kinetic dirichlet

In [15]:
##### We find the critical (z, k) eigenvalues between boundary and bulk
print("Bounce back")
pretty_print(solve([Lz.determinant().full_simplify(), bdMat_BB.determinant().full_simplify()], (z, k)))
print("Anti Bounce back")
pretty_print(solve([Lz.determinant().full_simplify(), bdMat_ABB.determinant().full_simplify()], (z, k)))
print("Two steps anti Bounce back")
pretty_print(solve([Lz.determinant().full_simplify(), bdMat_TwoABB.determinant().full_simplify()], (z, k)))
print("Extrapolation sigma = 1")
pretty_print(solve([Lz.determinant().full_simplify(), bdMat_sigma1.determinant().full_simplify()], (z, k)))
print("Extrapolation sigma = 2")
pretty_print(solve([Lz.determinant().full_simplify(), bdMat_sigma2.determinant().full_simplify()], (z, k)))
print("Extrapolation sigma = 3")
pretty_print(solve([Lz.determinant().full_simplify(), bdMat_sigma3.determinant().full_simplify()], (z, k)))
print("Extrapolation sigma = 4")
pretty_print(solve([Lz.determinant().full_simplify(), bdMat_sigma4.determinant().full_simplify()], (z, k)))
print("Kinetic Dirichlet")
pretty_print(solve([Lz.determinant().full_simplify(), bdMat_kinDir.determinant().full_simplify()], (z, k)))

Bounce back


[[z == 0, kappa == 0], [z == -1/2*(C - 1)*s2, kappa == 0], [z == 1, kappa == -((C - 1)*s2 + 2)/((C + 1)*s2 - 2)], [z == s2 - 1, kappa == -1]]

Anti Bounce back


[[z == 0, kappa == 0], [z == 1/2*(C - 1)*s2, kappa == 0], [z == -1, kappa == ((C - 1)*s2 + 2)/((C + 1)*s2 - 2)], [z == -s2 + 1, kappa == 1]]

Two steps anti Bounce back


...

[[z == 3/2*(C - 1)*s2 + 3, kappa == 0], [z == 0, kappa == 0], [z == 1, kappa == 1], [z == -s2 + 1, kappa == 1]]

Extrapolation sigma = 4


[[z == 2*(C - 1)*s2 + 4, kappa == 0], [z == 0, kappa == 0], [z == 1, kappa == 1], [z == -s2 + 1, kappa == 1]]

Kinetic Dirichlet


[[z == 0, kappa == 0]]

In [16]:
##### Sanity check : we set some values for s_2 and C and see if we find more
##### Sometimes sagemath does not give all the roots
s2_val = 2
C_val = -1/2
print("Bounce back")
pretty_print(solve([Lz.subs(s2==s2_val, C==C_val).determinant().full_simplify(), bdMat_BB.subs(s2==s2_val, C==C_val).determinant().full_simplify()], (z, k)))
print("Anti Bounce back")
pretty_print(solve([Lz.subs(s2==s2_val, C==C_val).determinant().full_simplify(), bdMat_ABB.subs(s2==s2_val, C==C_val).determinant().full_simplify()], (z, k)))
print("Two steps anti Bounce back")
pretty_print(solve([Lz.subs(s2==s2_val, C==C_val).determinant().full_simplify(), bdMat_TwoABB.subs(s2==s2_val, C==C_val).determinant().full_simplify()], (z, k)))
print("Extrapolation sigma = 1")
pretty_print(solve([Lz.subs(s2==s2_val, C==C_val).determinant().full_simplify(), bdMat_sigma1.subs(s2==s2_val, C==C_val).determinant().full_simplify()], (z, k)))
print("Extrapolation sigma = 2")
pretty_print(solve([Lz.subs(s2==s2_val, C==C_val).determinant().full_simplify(), bdMat_sigma2.subs(s2==s2_val, C==C_val).determinant().full_simplify()], (z, k)))
print("Extrapolation sigma = 3")
pretty_print(solve([Lz.subs(s2==s2_val, C==C_val).determinant().full_simplify(), bdMat_sigma3.subs(s2==s2_val, C==C_val).determinant().full_simplify()], (z, k)))
print("Extrapolation sigma = 4")
pretty_print(solve([Lz.subs(s2==s2_val, C==C_val).determinant().full_simplify(), bdMat_sigma4.subs(s2==s2_val, C==C_val).determinant().full_simplify()], (z, k)))
print("Kinetic Dirichlet")
pretty_print(solve([Lz.subs(s2==s2_val, C==C_val).determinant().full_simplify(), bdMat_kinDir.subs(s2==s2_val, C==C_val).determinant().full_simplify()], (z, k)))

Bounce back


[[z == 0, kappa == 0], [z == (3/2), kappa == 0], [z == 1, kappa == -1]]

Anti Bounce back


[[z == 0, kappa == 0], [z == (-3/2), kappa == 0], [z == -1, kappa == 1]]

Two steps anti Bounce back


...

[[z == (-3/2), kappa == 0], [z == 0, kappa == 0], [z == -1, kappa == 1], [z == 1, kappa == 1]]

Extrapolation sigma = 4


[[z == -2, kappa == 0], [z == 0, kappa == 0], [z == -1, kappa == 1], [z == 1, kappa == 1]]

Kinetic Dirichlet


[[z == 0, kappa == 0]]

In [17]:
#### Critical points (-1, -Pi) and (1, Pi) --- For C<0
assume(C<0)
print("Value of kminus at z=-1")
pretty_print(kminus.subs(z==-1).full_simplify())
print("Value of kplus at z=-1")
pretty_print(kplus.subs(z==-1).full_simplify())
print("Value of kminus at z=1")
pretty_print(kminus.subs(z==1).full_simplify())
print("Value of kplus at z=1")
pretty_print(kplus.subs(z==1).full_simplify())

print("Value of Pi")
pretty_print(Pi)

ks = kplus.full_simplify()
ku = kminus.full_simplify()


LzOnStable = Lz.subs(k==ks) ### This is the matrix polynomial where k is replaced by ks (stable root)
LzOnUnstable = Lz.subs(k==ku)

for i in range(q):
    for j in range(q):
        LzOnStable[i, j] = LzOnStable[i, j].full_simplify()
        LzOnUnstable[i, j] = LzOnUnstable[i, j].full_simplify()
        
#pretty_print(LzOnStable.transpose().kernel()) #### This is the stable eigenvalue phi_s
phiS = matrix([[LzOnStable.transpose().kernel().basis()[0][0]], \
               [LzOnStable.transpose().kernel().basis()[0][1].full_simplify()]])

phiU = matrix([[LzOnUnstable.transpose().kernel().basis()[0][0]], \
               [LzOnUnstable.transpose().kernel().basis()[0][1].full_simplify()]])

#### What are the values of z at which the second component of the eigenvector blows up
pretty_print(solve(phiS[1, 0].denominator().full_simplify(), z))
pretty_print(solve(phiS[1, 0].denominator().subs(s2==2).full_simplify(), z))
pretty_print(solve(phiU[1, 0].denominator().full_simplify(), z))
pretty_print(solve(phiU[1, 0].denominator().subs(s2==2).full_simplify(), z))
pretty_print(phiS[1, 0].numerator().subs(s2==2).full_simplify().subs(z==1))
pretty_print(phiS[1, 0].numerator().subs(s2==2).full_simplify().subs(z==-1))
pretty_print(phiU[1, 0].numerator().subs(s2==2).full_simplify().subs(z==1))
pretty_print(phiU[1, 0].numerator().subs(s2==2).full_simplify().subs(z==-1))

#### Structure of the eigenvector close to the critical points
print('Expansion of the second component of phiS around z = -1 as s2 != 2')
pretty_print(taylor(phiS[1][0], z, -1, 1))
print('Expansion of the second component of phiS around z = -1 as s2 = 2')
pretty_print(taylor(phiS[1][0].subs(s2=2), z, -1, 1))
print('Expansion of the second component of phiS around z = 1 as s2 != 2')
pretty_print(taylor(phiS[1][0], z, 1, 1))
print('Expansion of the second component of phiS around z = 1 as s2 = 2')
pretty_print(taylor(phiS[1][0].subs(s2=2), z, 1, 1))

#### We construct the eigenvector of k = 0
phi0 = matrix([[ (s2-1)*Lz.subs(k==0).transpose().kernel().basis()[0][0]], \
               [((s2-1)*Lz.subs(k==0).transpose().kernel().basis()[0][1]).full_simplify()]])

#### We start analyzing critical points and Kreiss Lopatinskii determinants

bd00  = matrix(SR, q, q)
bd01 = matrix(SR, q, q)
bd02 = matrix(SR, q, q)
bd03 = matrix(SR, q, q)
BKL = matrix(SR, q, q) #### Boundary matrix on the stable subspace

g1m1Tilde = 1 ### Source term used in the experiments

Value of kminus at z=-1


-1

Value of kplus at z=-1


((C - 1)*s2 + 2)/((C + 1)*s2 - 2)

Value of kminus at z=1


...

2*C/(z + 1) + 1/2*(z + 1)/C

Expansion of the second component of phiS around z = 1 as s2 != 2


C*s2/(s2 - 2) + ((C^2 - 1)*s2^2 + 4*s2 - 4)*(z - 1)/(C*s2^3 - 4*C*s2^2 + 4*C*s2)

Expansion of the second component of phiS around z = 1 as s2 = 2


-2*C/(z - 1) - 1/2*(z - 1)/C

In [ ]:
print("======= Bounce back ======")

for i in range(q):
    for j in range(q):
        bd00[i, j] = bdMat_BB[i, j].full_simplify().collect(k).coefficient(k, 0)
        bd01[i, j] = bdMat_BB[i, j].full_simplify().collect(k).coefficient(k, 1)
        
BKL[:, 0] = (bd00 + bd01*ks)*phiS ### First column
BKL[:, 1] = (bd00)*phi0 ### Second column

DetKL = BKL.determinant().full_simplify()

print("Kreiss Lopatinskii determinant when s2 = 2 at z = -1")
pretty_print(taylor(DetKL.subs(s2==2).full_simplify(), z, -1, -1))

print("Kreiss Lopatinskii determinant at z = 1")
pretty_print(taylor(DetKL, z, 1, 1))

print("Computation of the long-time solution for the source term in the experiments using residues")
var('j')
var('n')

Cs = (2+(C-1)*s2)*z*g1m1Tilde/DetKL

m1 = Cs*z**(n-1)*ks**(j)*phiS[0, 0]
m2 = Cs*z**(n-1)*ks**(j)*phiS[1, 0]

print("m1_j^n equals approximately (s2 in (0, 2))")
pretty_print(m1.maxima_methods().residue(z,-1) + m1.maxima_methods().residue(z,1))
print("m2_j^n equals approximately (s2 in (0, 2))")
pretty_print(m2.maxima_methods().residue(z,-1) + m2.maxima_methods().residue(z,1))

print("m1_j^n equals approximately (s2=2)")
pretty_print((m1.subs(s2==2).full_simplify().maxima_methods().residue(z,-1) \
            + m1.subs(s2==2).full_simplify().maxima_methods().residue(z,1)).factor())
print("m2_j^n equals approximately (s2=2)")
pretty_print((m2.subs(s2==2).full_simplify().maxima_methods().residue(z,-1) \
            + m2.subs(s2==2).full_simplify().maxima_methods().residue(z,1)).factor())


In [ ]:
print("======= Anti bounce back ======")

for i in range(q):
    for j in range(q):
        bd00[i, j] = bdMat_ABB[i, j].full_simplify().collect(k).coefficient(k, 0)
        bd01[i, j] = bdMat_ABB[i, j].full_simplify().collect(k).coefficient(k, 1)
        
BKL[:, 0] = (bd00 + bd01*ks)*phiS ### First column
BKL[:, 1] = (bd00)*phi0 ### Second column

DetKL = BKL.determinant().full_simplify()

print("Kreiss Lopatinskii determinant at z = -1")
pretty_print(taylor(DetKL, z, -1, 1))

print("Kreiss Lopatinskii determinant for s2 = 2 at z = 1")
pretty_print(taylor(DetKL.subs(s2==2).full_simplify(), z, 1, -1))

print("Computation of the long-time solution for the source term in the experiments using residues")
var('j')
var('n')

Cs = (2+(C-1)*s2)*z*g1m1Tilde/DetKL

m1 = Cs*z**(n-1)*ks**(j)*phiS[0, 0]
m2 = Cs*z**(n-1)*ks**(j)*phiS[1, 0]

print("m1_j^n equals approximately (s2 in (0, 2))")
pretty_print(m1.maxima_methods().residue(z,-1) + m1.maxima_methods().residue(z,1))
print("m2_j^n equals approximately (s2 in (0, 2))")
pretty_print(m2.maxima_methods().residue(z,-1) + m2.maxima_methods().residue(z,1))

print("m1_j^n equals approximately (s2=2)")
pretty_print((m1.subs(s2==2).full_simplify().maxima_methods().residue(z,-1) \
            + m1.subs(s2==2).full_simplify().maxima_methods().residue(z,1)).factor())
print("m2_j^n equals approximately (s2=2)")
pretty_print((m2.subs(s2==2).full_simplify().maxima_methods().residue(z,-1) \
            + m2.subs(s2==2).full_simplify().maxima_methods().residue(z,1)).factor())


In [ ]:
print("======= Two-steps anti bounce back ======")

for i in range(q):
    for j in range(q):
        bd00[i, j] = bdMat_TwoABB[i, j].full_simplify().collect(k).coefficient(k, 0)
        bd01[i, j] = bdMat_TwoABB[i, j].full_simplify().collect(k).coefficient(k, 1)
        
BKL[:, 0] = (bd00 + bd01*ks)*phiS ### First column
BKL[:, 1] = (bd00)*phi0 ### Second column

DetKL = BKL.determinant().full_simplify()

print("Kreiss Lopatinskii determinant for s2 = 2 at z = -1")
pretty_print(taylor(DetKL.subs(s2==2).full_simplify(), z, -1, 0))

print("First column of BKL when s2 = 2 at z = -1")
pretty_print(taylor(BKL[:, 0].subs(s2==2), z, -1, 0))

print("Kreiss Lopatinskii determinant for s2 = 2 at z = 1")
pretty_print(taylor(DetKL.subs(s2==2).full_simplify(), z, 1, 0))

print("Computation of the long-time solution for the source term in the experiments using residues")
var('j')
var('n')

Cs = (2+(C-1)*s2)*z*g1m1Tilde/DetKL

m1 = Cs*z**(n-1)*ks**(j)*phiS[0, 0]
m2 = Cs*z**(n-1)*ks**(j)*phiS[1, 0]

print("m1_j^n equals approximately (s2 in (0, 2))")
pretty_print(m1.maxima_methods().residue(z,-1) + m1.maxima_methods().residue(z,1))
print("m2_j^n equals approximately (s2 in (0, 2))")
pretty_print(m2.maxima_methods().residue(z,-1) + m2.maxima_methods().residue(z,1))

print("m1_j^n equals approximately (s2=2)")
pretty_print((m1.subs(s2==2).full_simplify().maxima_methods().residue(z,-1) \
            + m1.subs(s2==2).full_simplify().maxima_methods().residue(z,1)).factor())
print("m2_j^n equals approximately (s2=2)")
pretty_print((m2.subs(s2==2).full_simplify().maxima_methods().residue(z,-1) \
            + m2.subs(s2==2).full_simplify().maxima_methods().residue(z,1)).factor())


In [ ]:
print("======= Extrapolation sigma = 1 ======")

for i in range(q):
    for j in range(q):
        bd00[i, j] = bdMat_sigma1[i, j].full_simplify().collect(k).coefficient(k, 0)
        bd01[i, j] = bdMat_sigma1[i, j].full_simplify().collect(k).coefficient(k, 1)
        
BKL[:, 0] = (bd00 + bd01*ks)*phiS ### First column
BKL[:, 1] = (bd00)*phi0 ### Second column

DetKL = BKL.determinant().full_simplify()

print("Kreiss Lopatinskii determinant for s2 = 2 at z = -1")
pretty_print(taylor(DetKL.subs(s2==2).full_simplify(), z, -1, 0))

print("First column of BKL when s2 = 2 at z = -1")
pretty_print(taylor(BKL[:, 0].subs(s2==2), z, -1, 0))

print("Kreiss Lopatinskii determinant for s2 = 2 at z = 1")
pretty_print(taylor(DetKL.subs(s2==2).full_simplify(), z, 1, 0))

print("Computation of the long-time solution for the source term in the experiments using residues")
var('j')
var('n')

Cs = (2+(C-1)*s2)*z*g1m1Tilde/DetKL

m1 = Cs*z**(n-1)*ks**(j)*phiS[0, 0]
m2 = Cs*z**(n-1)*ks**(j)*phiS[1, 0]

print("m1_j^n equals approximately (s2 in (0, 2))")
pretty_print(m1.maxima_methods().residue(z,-1) + m1.maxima_methods().residue(z,1))
print("m2_j^n equals approximately (s2 in (0, 2))")
pretty_print(m2.maxima_methods().residue(z,-1) + m2.maxima_methods().residue(z,1))

print("m1_j^n equals approximately (s2=2)")
pretty_print((m1.subs(s2==2).full_simplify().maxima_methods().residue(z,-1) \
            + m1.subs(s2==2).full_simplify().maxima_methods().residue(z,1)).factor())
print("m2_j^n equals approximately (s2=2)")
pretty_print((m2.subs(s2==2).full_simplify().maxima_methods().residue(z,-1) \
            + m2.subs(s2==2).full_simplify().maxima_methods().residue(z,1)).factor())


In [ ]:
print("======= Extrapolation sigma = 2 ======")

for i in range(q):
    for j in range(q):
        bd00[i, j] = bdMat_sigma2[i, j].full_simplify().collect(k).coefficient(k, 0)
        bd01[i, j] = bdMat_sigma2[i, j].full_simplify().collect(k).coefficient(k, 1)
        
BKL[:, 0] = (bd00 + bd01*ks)*phiS ### First column
BKL[:, 1] = (bd00)*phi0 ### Second column

DetKL = BKL.determinant().full_simplify()

print("Kreiss Lopatinskii determinant for s2 = 2 at z = -1")
pretty_print(taylor(DetKL.subs(s2==2).full_simplify(), z, -1, 1))

print("First column of BKL when s2 = 2 at z = -1")
pretty_print(taylor(BKL[:, 0].subs(s2==2), z, -1, 1))

print("Kreiss Lopatinskii determinant for s2 = 2 at z = 1")
pretty_print(taylor(DetKL.subs(s2==2).full_simplify(), z, 1, -1))

print("Computation of the long-time solution for the source term in the experiments using residues")
var('j')
var('n')

Cs = (2+(C-1)*s2)*z*g1m1Tilde/DetKL

m1 = Cs*z**(n-1)*ks**(j)*phiS[0, 0]
m2 = Cs*z**(n-1)*ks**(j)*phiS[1, 0]

print("m1_j^n equals approximately (s2=2)")
pretty_print((m1.subs(s2==2).full_simplify().maxima_methods().residue(z,-1) \
            + m1.subs(s2==2).full_simplify().maxima_methods().residue(z,1)).factor())
print("m2_j^n equals approximately (s2=2)")
pretty_print((m2.subs(s2==2).full_simplify().maxima_methods().residue(z,-1) \
            + m2.subs(s2==2).full_simplify().maxima_methods().residue(z,1)).factor())

In [ ]:
print("======= Extrapolation sigma = 3 ======")

for i in range(q):
    for j in range(q):
        bd00[i, j] = bdMat_sigma3[i, j].full_simplify().collect(k).coefficient(k, 0)
        bd01[i, j] = bdMat_sigma3[i, j].full_simplify().collect(k).coefficient(k, 1)
        bd02[i, j] = bdMat_sigma3[i, j].full_simplify().collect(k).coefficient(k, 2)
        
BKL[:, 0] = (bd00 + bd01*ks + bd02*ks**2)*phiS ### First column
BKL[:, 1] = (bd00)*phi0 ### Second column

DetKL = BKL.determinant().full_simplify()

print("Kreiss Lopatinskii determinant for s2 = 2 at z = -1")
pretty_print(taylor(DetKL.subs(s2==2).full_simplify(), z, -1, 2))

print("First column of BKL when s2 = 2 at z = -1")
pretty_print(taylor(BKL[:, 0].subs(s2==2), z, -1, 2))

print("Kreiss Lopatinskii determinant for s2 = 2 at z = 1")
pretty_print(taylor(DetKL.subs(s2==2).full_simplify(), z, 1, -1))

print("Computation of the long-time solution for the source term in the experiments using residues")
var('j')
var('n')

Cs = (2+(C-1)*s2)*z*g1m1Tilde/DetKL

m1 = Cs*z**(n-1)*ks**(j)*phiS[0, 0]
m2 = Cs*z**(n-1)*ks**(j)*phiS[1, 0]

print("m1_j^n equals approximately (s2=2)")
pretty_print((m1.subs(s2==2).full_simplify().maxima_methods().residue(z,-1) \
            + m1.subs(s2==2).full_simplify().maxima_methods().residue(z,1)).factor())
print("m2_j^n equals approximately (s2=2)")
pretty_print((m2.subs(s2==2).full_simplify().maxima_methods().residue(z,-1) \
            + m2.subs(s2==2).full_simplify().maxima_methods().residue(z,1)).factor())

In [ ]:
print("======= Extrapolation sigma = 4 ======")

for i in range(q):
    for j in range(q):
        bd00[i, j] = bdMat_sigma4[i, j].full_simplify().collect(k).coefficient(k, 0)
        bd01[i, j] = bdMat_sigma4[i, j].full_simplify().collect(k).coefficient(k, 1)
        bd02[i, j] = bdMat_sigma4[i, j].full_simplify().collect(k).coefficient(k, 2)
        bd03[i, j] = bdMat_sigma4[i, j].full_simplify().collect(k).coefficient(k, 3)
        
BKL[:, 0] = (bd00 + bd01*ks + bd02*ks**2 + bd03*ks**3)*phiS ### First column
BKL[:, 1] = (bd00)*phi0 ### Second column

DetKL = BKL.determinant().full_simplify()

print("Kreiss Lopatinskii determinant for s2 = 2 at z = -1")
pretty_print(taylor(DetKL.subs(s2==2).full_simplify(), z, -1, 3))

print("First column of BKL when s2 = 2 at z = -1")
pretty_print(taylor(BKL[:, 0].subs(s2==2), z, -1, 3))

print("Kreiss Lopatinskii determinant for s2 = 2 at z = 1")
pretty_print(taylor(DetKL.subs(s2==2).full_simplify(), z, 1, -1))

print("Computation of the long-time solution for the source term in the experiments using residues")
var('j')
var('n')

Cs = (2+(C-1)*s2)*z*g1m1Tilde/DetKL

m1 = Cs*z**(n-1)*ks**(j)*phiS[0, 0]
m2 = Cs*z**(n-1)*ks**(j)*phiS[1, 0]

print("m1_j^n equals approximately (s2=2)")
pretty_print((m1.subs(s2==2).full_simplify().maxima_methods().residue(z,-1) \
            + m1.subs(s2==2).full_simplify().maxima_methods().residue(z,1)).factor())
print("m2_j^n equals approximately (s2=2)")
pretty_print((m2.subs(s2==2).full_simplify().maxima_methods().residue(z,-1) \
            + m2.subs(s2==2).full_simplify().maxima_methods().residue(z,1)).factor())

In [ ]:
print("======= Kinetic Dirichlet ======")

for i in range(q):
    for j in range(q):
        bd00[i, j] = bdMat_kinDir[i, j].full_simplify().collect(k).coefficient(k, 0)
        bd01[i, j] = bdMat_kinDir[i, j].full_simplify().collect(k).coefficient(k, 1)
        
BKL[:, 0] = (bd00 + bd01*ks)*phiS ### First column
BKL[:, 1] = (bd00)*phi0 ### Second column

DetKL = BKL.determinant().full_simplify()

print("Kreiss Lopatinskii determinant for s2 = 2 at z = -1")
pretty_print(taylor(DetKL.subs(s2==2).full_simplify(), z, -1, -1))

print("Kreiss Lopatinskii determinant for s2 = 2 at z = 1")
pretty_print(taylor(DetKL.subs(s2==2).full_simplify(), z, 1, -1))

print("Computation of the long-time solution for the source term in the experiments using residues")
var('j')
var('n')

Cs = (2+(C-1)*s2)*z*g1m1Tilde/DetKL

m1 = Cs*z**(n-1)*ks**(j)*phiS[0, 0]
m2 = Cs*z**(n-1)*ks**(j)*phiS[1, 0]

print("m1_j^n equals approximately (s2=2)")
pretty_print((m1.subs(s2==2).full_simplify().maxima_methods().residue(z,-1) \
            + m1.subs(s2==2).full_simplify().maxima_methods().residue(z,1)).factor())
print("m2_j^n equals approximately (s2=2)")
pretty_print((m2.subs(s2==2).full_simplify().maxima_methods().residue(z,-1) \
            + m2.subs(s2==2).full_simplify().maxima_methods().residue(z,1)).factor())

In [ ]:
forget()

#### We make natural assumptions
assume(s2>0)
assume(s2<=2)
#### Critical points (1, 1) --- For C>0 
assume(C>0)

print("Value of kminus at z=1")
pretty_print(kminus.subs(z==1).full_simplify())
print("Value of kplus at z=1")
pretty_print(kplus.subs(z==1).full_simplify())

ks = kplus.full_simplify()

LzOnStable = Lz.subs(k==ks) ### This is the matrix polynomial where k is replaced by ks (stable root)
for i in range(q):
    for j in range(q):
        LzOnStable[i, j] = LzOnStable[i, j].full_simplify()
        
#pretty_print(LzOnStable.transpose().kernel()) #### This is the stable eigenvalue phi_s
phiS = matrix([[LzOnStable.transpose().kernel().basis()[0][0]], \
               [LzOnStable.transpose().kernel().basis()[0][1].full_simplify()]])

#### Structure of the eigenvector close to the critical points
print('Expansion of the second component of phiS around z = 1')
pretty_print(taylor(phiS[1][0], z, 1, 1)) # I do not know why taking .subs(z==1) gives wrong results
print('Expansion of the second component of phiS around z = 1 as s2 = 2')
pretty_print(taylor(phiS[1][0].subs(s2=2), z, 1, 1))
print('Expansion of the second component of phiS around z = -1 as s2 = 2')
pretty_print(taylor(phiS[1][0].subs(s2=2), z, -1, 1))

#### We construct the eigenvector of k = 0
phi0 = matrix([[ (s2-1)*Lz.subs(k==0).transpose().kernel().basis()[0][0]], \
               [((s2-1)*Lz.subs(k==0).transpose().kernel().basis()[0][1]).full_simplify()]])


In [ ]:
print("======= Extrapolation sigma = 1 ======")

for i in range(q):
    for j in range(q):
        bd00[i, j] = bdMat_sigma1[i, j].full_simplify().collect(k).coefficient(k, 0)
        bd01[i, j] = bdMat_sigma1[i, j].full_simplify().collect(k).coefficient(k, 1)
        
BKL[:, 0] = (bd00 + bd01*ks)*phiS ### First column
BKL[:, 1] = (bd00)*phi0 ### Second column

DetKL = BKL.determinant().full_simplify()

print("Kreiss Lopatinskii determinant at z = 1")
pretty_print(taylor(DetKL.full_simplify(), z, 1, 1))

print("Computation of the long-time solution for the source term in the experiments using residues")
var('j')
var('n')

Cs = (2+(C-1)*s2)*z*g1m1Tilde/DetKL

m1 = Cs*z**(n-1)*ks**(j)*phiS[0, 0]
m2 = Cs*z**(n-1)*ks**(j)*phiS[1, 0]

print("m1_j^n equals approximately (s2 in (0, 2])")
pretty_print(m1.maxima_methods().residue(z,-1) + m1.maxima_methods().residue(z,1))
print("m2_j^n equals approximately (s2 in (0, 2])")
pretty_print(m2.maxima_methods().residue(z,-1) + m2.maxima_methods().residue(z,1))


In [ ]:
print("======= Extrapolation sigma = 2 ======")

for i in range(q):
    for j in range(q):
        bd00[i, j] = bdMat_sigma2[i, j].full_simplify().collect(k).coefficient(k, 0)
        bd01[i, j] = bdMat_sigma2[i, j].full_simplify().collect(k).coefficient(k, 1)
        
BKL[:, 0] = (bd00 + bd01*ks)*phiS ### First column
BKL[:, 1] = (bd00)*phi0 ### Second column

DetKL = BKL.determinant().full_simplify()

print("Kreiss Lopatinskii determinant at z = 1")
pretty_print(taylor(DetKL.full_simplify(), z, 1, 2))

print("Computation of the long-time solution for the source term in the experiments using residues")
var('j')
var('n')

Cs = (2+(C-1)*s2)*z*g1m1Tilde/DetKL

m1 = Cs*z**(n-1)*ks**(j)*phiS[0, 0]
m2 = Cs*z**(n-1)*ks**(j)*phiS[1, 0]

print("m1_j^n equals approximately (s2 in (0, 2])")
pretty_print(m1.maxima_methods().residue(z,1).factor())
print("m2_j^n equals approximately (s2 in (0, 2])")
pretty_print(m2.maxima_methods().residue(z,1).factor())


In [ ]:
print("======= Extrapolation sigma = 3 ======")

for i in range(q):
    for j in range(q):
        bd00[i, j] = bdMat_sigma3[i, j].full_simplify().collect(k).coefficient(k, 0)
        bd01[i, j] = bdMat_sigma3[i, j].full_simplify().collect(k).coefficient(k, 1)
        bd02[i, j] = bdMat_sigma3[i, j].full_simplify().collect(k).coefficient(k, 2)
        
BKL[:, 0] = (bd00 + bd01*ks + bd02*ks**2)*phiS ### First column
BKL[:, 1] = (bd00)*phi0 ### Second column

DetKL = BKL.determinant().full_simplify()

print("Kreiss Lopatinskii determinant at z = 1")
pretty_print(taylor(DetKL.full_simplify(), z, 1, 3))

print("Computation of the long-time solution for the source term in the experiments using residues")
var('j')
var('n')

Cs = (2+(C-1)*s2)*z*g1m1Tilde/DetKL

m1 = Cs*z**(n-1)*ks**(j)*phiS[0, 0]
m2 = Cs*z**(n-1)*ks**(j)*phiS[1, 0]

print("m1_j^n equals approximately (s2 in (0, 2])")
pretty_print(m1.maxima_methods().residue(z,1).factor())
print("m2_j^n equals approximately (s2 in (0, 2])")
pretty_print(m2.maxima_methods().residue(z,1).factor())


In [ ]:
print("======= Extrapolation sigma = 4 ======")

for i in range(q):
    for j in range(q):
        bd00[i, j] = bdMat_sigma4[i, j].full_simplify().collect(k).coefficient(k, 0)
        bd01[i, j] = bdMat_sigma4[i, j].full_simplify().collect(k).coefficient(k, 1)
        bd02[i, j] = bdMat_sigma4[i, j].full_simplify().collect(k).coefficient(k, 2)
        bd03[i, j] = bdMat_sigma4[i, j].full_simplify().collect(k).coefficient(k, 3)
        
BKL[:, 0] = (bd00 + bd01*ks + bd02*ks**2 + bd03*ks**3)*phiS ### First column
BKL[:, 1] = (bd00)*phi0 ### Second column

DetKL = BKL.determinant().full_simplify()

print("Kreiss Lopatinskii determinant at z = 1")
pretty_print(taylor(DetKL.full_simplify(), z, 1, 4))

print("Computation of the long-time solution for the source term in the experiments using residues")
var('j')
var('n')

Cs = (2+(C-1)*s2)*z*g1m1Tilde/DetKL

m1 = Cs*z**(n-1)*ks**(j)*phiS[0, 0]
m2 = Cs*z**(n-1)*ks**(j)*phiS[1, 0]

print("m1_j^n equals approximately (s2 in (0, 2])")
pretty_print(m1.maxima_methods().residue(z,1).factor())
print("m2_j^n equals approximately (s2 in (0, 2])")
pretty_print(m2.maxima_methods().residue(z,1).factor())


In [ ]:
#### We now consider specific values of C and s2, for those boundary conditions which we have not considered so far

bdMat_Ext_Eq_sigma1 = z*identity_matrix(q) - M*matrix([[1/2*(1+C), 1/2*(1+C)], [0, k]])*Minv*K ## Extr eq. sigma=1
bdMat_Ext_Eq_sigma3 = z*identity_matrix(q) - \
          M*matrix([[1/2*(1+C)*(3-3*k+k**2), 1/2*(1+C)*(3-3*k+k**2)], [0, k]])*Minv*K ## Extr eq. sigma=3
bdMat_future = z*identity_matrix(q) - M*matrix([[1/2*(1+C), 1/2*(1+C)*k**2], [0, k]])*Minv*K ## Future
bdMat_invented = z*identity_matrix(q) - M*matrix([[1, 2], [0, k]])*Minv*K ## Invented


for param in [(3/2, -1/2), (2, -1/2), (3/2, 1/2), (2, 1/2)]:
#for param in [(3/2, -1/2)]:

    s2Val = param[0]
    CVal  = param[1]
    
    print('======== s2 = '+str(s2Val)+'   C = '+str(CVal)+' ========')
    
    print('Extrapolated equilibrium sigma = 1')
    roots = solve([charEq.subs(s2==s2Val, C==CVal).full_simplify(), \
                   bdMat_Ext_Eq_sigma1.subs(s2==s2Val, C==CVal).determinant().full_simplify()], [z, k])
    for root in roots:
        pretty_print(root)
        print('abs of z = '+str(abs(root[0].rhs()).n())+'   abs of k = '+str(abs(root[1].rhs()).n()))
        
    print('------')
    print('Extrapolated equilibrium sigma = 3')
    roots = solve([charEq.subs(s2==s2Val, C==CVal).full_simplify(), \
                   bdMat_Ext_Eq_sigma3.subs(s2==s2Val, C==CVal).determinant().full_simplify()], [z, k])
    for root in roots:
        pretty_print(root)
        print('abs of z = '+str(abs(root[0].rhs()).n())+'   abs of k = '+str(abs(root[1].rhs()).n()))
    
    print('------')
    print('Future')
    roots = solve([charEq.subs(s2==s2Val, C==CVal).full_simplify(), \
                   bdMat_future.subs(s2==s2Val, C==CVal).determinant().full_simplify()], [z, k])
    for root in roots:
        pretty_print(root)
        print('abs of z = '+str(abs(root[0].rhs()).n())+'   abs of k = '+str(abs(root[1].rhs()).n()))
        
    print('------')
    print('Invented')
    roots = solve([charEq.subs(s2==s2Val, C==CVal).full_simplify(), \
                   bdMat_invented.subs(s2==s2Val, C==CVal).determinant().full_simplify()], [z, k])
    for root in roots:
        pretty_print(root)
        print('abs of z = '+str(abs(root[0].rhs()).n())+'   abs of k = '+str(abs(root[1].rhs()).n()))

In [43]:
pretty_print(phiS[1, 0].numerator().subs(s2==2).full_simplify())

z^4 + 2*(2*C^2 + C - 1)*z^2 - sqrt(z^4 + 2*(2*C^2 - 1)*z^2 + 1)*(z^2 + 2*C - 1) - 2*C + 1

In [23]:
pretty_print(phiS[1, 0].numerator().subs(s2==2, z=-1))

8*C^2

In [41]:
pretty_print((Lz[0, 0]/Lz[0, 1]).subs(k==ks).subs(s2==2).full_simplify())

(z^4 + 2*(2*C^2 + C - 1)*z^2 - sqrt(z^4 + 2*(2*C^2 - 1)*z^2 + 1)*(z^2 + 2*C - 1) - 2*C + 1)/(z^4 - 2*z^2 - sqrt(z^4 + 2*(2*C^2 - 1)*z^2 + 1)*(z^2 - 1) + 1)